![](Images/image.png)

# Optimizations Databricks

There are plenty of optimizations that Databricks does by default or that are required to be enabled. This notebook shows some of them

## Databricks Performance
### Modern Optimizations vs. Legacy Methods

This notebook benchmarks the performance of Databricks modern features (Liquid Clustering, Photon, Deletion Vectors) against legacy strategies (Partitioning + Z-Order) and unoptimized baselines.

**Cluster Config:** 2 Workers (Fixed), Photon Enabled.

**Test Scenarios:**
1.  **Data Layout:** Standard vs. Partitioning+ZOrder vs. Liquid Clustering.
2.  **DML Operations:** Copy-on-Write vs. Deletion Vectors.
3.  **Read Speed:** Cold Read vs. Disk Cache (Hot).
4.  **ETL Merge:** Standard Shuffle vs. Low Shuffle Merge.
5.  **Search:** Full Scan vs. Bloom Filters.
6.  **DevOps:** Deep Clone vs. Shallow Clone.

## Setup

In [0]:
import time
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# --- CONFIGURATION ---
target_catalog = "catalog_databricks"
target_schema  = "schema"  # The specific schema you requested

username = spark.sql("SELECT current_user()").collect()[0][0].split('@')[0]
base_path = f"dbfs:/tmp/{username}/benchmark_clickstream_source"

print(f"🚀 Starting Unity Catalog Setup...")
print(f"📍 Catalog: {target_catalog}")
print(f"📍 Schema:   {target_schema}")

# 1. Schema Setup (Clean & Create)
try:
    spark.sql(f"USE CATALOG {target_catalog}")
    # WARNING: DROP CASCADE deletes everything in this schema to ensure a clean test
    spark.sql(f"DROP SCHEMA IF EXISTS {target_schema} CASCADE")
    spark.sql(f"CREATE SCHEMA {target_schema}")
    spark.sql(f"USE SCHEMA {target_schema}")
    print(f"✅ Schema '{target_schema}' created and selected.")
except Exception as e:
    print(f"❌ Error manipulating schema. Check permissions.")
    raise e

# 2. Benchmark Functions
results = []

def run_benchmark(test_name, method, func):
    print(f"🚀 Running: {test_name} [{method}]...")
    spark.catalog.clearCache() 
    start_ts = time.time()
    func()
    end_ts = time.time()
    duration = end_ts - start_ts
    print(f"   ⏱️ Time: {duration:.4f} seconds\n")
    results.append({"Test": test_name, "Method": method, "Duration_Sec": duration})

def display_results(prefix):
    section_data = [r for r in results if r['Test'].startswith(prefix)]
    if not section_data: return
    
    df_pd = pd.DataFrame(section_data)
    
    # Baseline Logic
    def get_baseline_duration(test_name):
        baseline_row = df_pd[
            (df_pd['Test'] == test_name) & 
            (df_pd['Method'].str.contains("Standard") | df_pd['Method'].str.contains("Legacy") | df_pd['Method'].str.contains("Cold"))
        ]
        if not baseline_row.empty: return baseline_row.iloc[0]['Duration_Sec']
        return df_pd[df_pd['Test'] == test_name]['Duration_Sec'].max()

    df_pd['Baseline'] = df_pd['Test'].apply(get_baseline_duration)
    df_pd['Speedup'] = df_pd['Baseline'] / df_pd['Duration_Sec']
    
    # Ranking Logic
    def get_rank(method_name):
        if "Standard" in method_name or "Legacy" in method_name: return 1
        return 2
    
    df_pd['Rank'] = df_pd['Method'].apply(get_rank)
    df_pd['Speedup (vs Std)'] = df_pd['Speedup'].apply(lambda x: f"{x:.1f}x")
    df_pd['Duration_Sec'] = df_pd['Duration_Sec'].apply(lambda x: round(x, 4))
    
    final_df = df_pd.sort_values(by=['Test', 'Rank'])[['Test', 'Method', 'Duration_Sec', 'Speedup (vs Std)']]
    
    print(f"--- 📊 Partial Results ({prefix}) ---")
    display(spark.createDataFrame(final_df))

def get_dir_size_mb(path):
    try:
        files = dbutils.fs.ls(path)
        total_bytes = sum([f.size for f in files if f.name.endswith(".parquet") or f.name.endswith(".csv")])
        return total_bytes / (1024 * 1024)
    except:
        return 0.0

# --- DATA GENERATION ---
num_rows = 15000000 
print(f"Generating {num_rows/1000000}M clickstream events...")

df_source = spark.range(0, num_rows) \
    .withColumn("user_id", (F.col("id") % 500000).cast(IntegerType())) \
    .withColumn("session_id", F.rand() * 1000000) \
    .withColumn("url", F.lit("/checkout/confirmation")) \
    .withColumn("event_date", F.date_add(F.lit("2024-01-01"), (F.col("id") % 30).cast(IntegerType()))) \
    .withColumn("event_hour", (F.col("id") % 24).cast(IntegerType()))

# --- STORAGE EFFICIENCY CHECK ---
print("Calculating Storage Efficiency...")

# 1. RAW Size (Uncompressed, simulating bad CSV/Parquet)
raw_path = f"{base_path}/check_raw_uncompressed"
dbutils.fs.rm(raw_path, True) # Clean if exists
df_source.write.format("parquet").option("compression", "uncompressed").mode("overwrite").save(raw_path)
raw_size_mb = get_dir_size_mb(raw_path)

# 2. Optimized Size (Delta Lake with Snappy/ZSTD)
# Save a temp table just to check managed size
spark.sql("DROP TABLE IF EXISTS tbl_size_check")
df_source.write.format("delta").mode("overwrite").saveAsTable("tbl_size_check")

# Get real Delta table size from metadata
delta_size_bytes = spark.sql("DESCRIBE DETAIL tbl_size_check").select("sizeInBytes").collect()[0][0]
opt_size_mb = delta_size_bytes / (1024 * 1024)

# Drop check table
spark.sql("DROP TABLE tbl_size_check")

savings = raw_size_mb / opt_size_mb if opt_size_mb > 0 else 0

# Cache to speed up next steps
df_source.cache() 
df_source.count() # Force cache materialization

print(f"\n" + "="*50)
print(f"--- Data Volume & Compression Analysis ---")
print(f"Rows Generated           : {num_rows:,}")
print(f"Raw Size (Uncompressed)  : {raw_size_mb:.2f} MB")
print(f"Delta Size (Compressed)  : {opt_size_mb:.2f} MB")
print(f"Storage Efficiency       : {savings:.1f}x Smaller")
print(f"Schema Used              : {target_catalog}.{target_schema}")
print("="*50 + "\n")

## Data Layout

In [0]:
target_user = 42000

# ==========================================
# 1. Standard (Partitioned only - The Baseline)
# ==========================================
def write_standard():
    # Writes data partitioned by Date/Hour.
    # This is the default behavior for most Data Lakes.
    # It creates ~720 directories.
    df_source.write.format("delta").mode("overwrite") \
        .partitionBy("event_date", "event_hour") \
        .saveAsTable("tbl_standard")

run_benchmark("1. Ingestion", "Standard (Part Only)", write_standard)

def query_standard():
    # Problem: Spark must open 720 directories.
    # Inside each directory, it has to scan the whole file because there is no Z-Order.
    spark.read.table("tbl_standard").filter(f"user_id = {target_user}").count()

run_benchmark("1. Read (Filter)", "Standard (Slow Scan)", query_standard)


# ==========================================
# 2. Legacy (Partitioned + Z-Order - The "Tax")
# ==========================================
def write_legacy_zorder():
    # Step 1: Write Partitioned (Same as Standard)
    df_source.write.format("delta").mode("overwrite") \
        .partitionBy("event_date", "event_hour") \
        .saveAsTable("tbl_legacy_zorder")
    
    # Step 2: The "Hidden Tax". We runs Z-ORDER to fix the read speed.
    # This reads and rewrites the data again!
    spark.sql("OPTIMIZE tbl_legacy_zorder ZORDER BY (user_id)")

run_benchmark("1. Ingestion", "Legacy (Part + ZOrder)", write_legacy_zorder)

def query_legacy_zorder():
    # Z-Order helps skip data WITHIN the files, but Spark still lists 720 directories.
    spark.read.table("tbl_legacy_zorder").filter(f"user_id = {target_user}").count()

run_benchmark("1. Read (Filter)", "Legacy (ZOrder Read)", query_legacy_zorder)


# ==========================================
# 3. Liquid Clustering (The Solution)
# ==========================================
def write_liquid():
    # Liquid handles everything in ONE pass.
    # No partitions to manage. No separate Z-Order command.
    df_source.write.format("delta") \
        .mode("overwrite") \
        .option("liquidClustering", "true") \
        .clusterBy("user_id") \
        .saveAsTable("tbl_liquid")

run_benchmark("1. Ingestion", "Liquid Clustering", write_liquid)

def query_liquid():
    # Liquid goes straight to the file containing the user.
    # Zero directory listing overhead.
    spark.read.table("tbl_liquid").filter(f"user_id = {target_user}").count()

run_benchmark("1. Read (Filter)", "Liquid Clustering", query_liquid)

display_results("1.")

In [0]:
display(spark.sql(f"SELECT * FROM catalog_databricks.schema.tbl_standard LIMIT 10"))

## DML Operations

In [0]:
import pyspark.sql.functions as F

# --- AUTO-CLEANUP ---
# Remove results from previous runs of Section 2 to avoid duplicate rows
global results
results = [r for r in results if not r['Test'].startswith("2.")]

# ==========================================
# DIMENSION 2: DML OPERATIONS (Delete/Update)
# Strategy: Create a Single Giant File (~1GB).
# CoW must rewrite the whole GB. DV touches nothing.
# ==========================================

print("Preparing High-Volume Data for DML Test...")

# 1. READ & BULK UP
# We take the previous data and add a 'payload' column to make it physically heavy.
# This ensures the file is large enough (~500MB+) to make Copy-on-Write painful.
df_heavy = spark.read.table("tbl_liquid") \
    .withColumn("payload", F.expr("repeat('DATA_LAKE_SCALE_', 50)")) \
    .repartition(1) # FORCE 1 SINGLE GIANT FILE

# Materialize this "Giant File" table
spark.sql("DROP TABLE IF EXISTS tbl_dml_source")
df_heavy.write.format("delta").mode("overwrite").saveAsTable("tbl_dml_source")

print("Tables prepared. Creating competitors...")

# 2. Setup Competitors
spark.sql("DROP TABLE IF EXISTS tbl_cow")
spark.sql("DROP TABLE IF EXISTS tbl_dv")

# A: Copy-on-Write Table (Legacy / Standard)
spark.sql("CREATE TABLE tbl_cow CLONE tbl_dml_source")
spark.sql("ALTER TABLE tbl_cow SET TBLPROPERTIES ('delta.enableDeletionVectors' = false)")

# B: Deletion Vectors Table (Modern)
spark.sql("CREATE TABLE tbl_dv CLONE tbl_dml_source")
spark.sql("ALTER TABLE tbl_dv SET TBLPROPERTIES ('delta.enableDeletionVectors' = true)")


# --- TEST A: COPY-ON-WRITE (Standard) ---
def delete_cow():
    # Scenario: Delete sparse records (approx 0.1% of data)
    # The Penalty: Spark must Read 1GB -> Filter -> Rewrite 0.99GB.
    spark.sql("DELETE FROM tbl_cow WHERE id % 1000 = 999")

run_benchmark("2. Delete", "Copy-on-Write (Standard)", delete_cow)


# --- TEST B: DELETION VECTORS (Modern) ---
def delete_dv():
    # The Gain: Spark marks rows as deleted in a tiny metadata file.
    # The 1GB file is NOT rewritten.
    spark.sql("DELETE FROM tbl_dv WHERE id % 1000 = 999")

run_benchmark("2. Delete", "Deletion Vectors (Modern)", delete_dv)


# --- SHOW COMPARISON ---
display_results("2.")

## Read Speed

In [0]:
# --- AUTO-CLEANUP ---
global results
results = [r for r in results if not r['Test'].startswith("3.")]

# ==========================================
# DIMENSION 3: DISK CACHE (Photon NVMe)
# Scenario: Heavy Aggregation on the Liquid Table.
# Cold: Reading from Object Storage (S3/ADLS).
# Hot: Reading from Local SSD (No Network I/O).
# ==========================================

target_table = "tbl_liquid"

# Define a heavy query (Aggregating 15M rows)
def complex_aggregation():
    spark.read.table(target_table) \
        .groupBy("event_date") \
        .agg(F.sum("id"), F.count("*"), F.max("session_id")) \
        .collect()

# --- TEST A: COLD READ (Standard / No Cache) ---
# We force clear the cache to simulate a fresh cluster or new data.
spark.catalog.clearCache()
spark.conf.set("spark.databricks.io.cache.enabled", "false")

run_benchmark("3. Caching", "Cold Read (Standard)", complex_aggregation)


# --- WARMUP PHASE (Hidden) ---
# We enable cache and run the query once to populate the SSDs.
# We don't time this for the graph, but you can mention it takes the same time as Cold Read.
print("🔥 Warming up the cache...")
spark.conf.set("spark.databricks.io.cache.enabled", "true")
complex_aggregation() 


# --- TEST B: HOT READ (Optimized) ---
# Now data is local. This simulates the experience of interactive users / BI Dashboards.
run_benchmark("3. Caching", "Hot Read (Disk Cache)", complex_aggregation)

# --- SHOW COMPARISON ---
display_results("3.")

## ETL Merge

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# --- AUTO-CLEANUP ---
global results
results = [r for r in results if not r['Test'].startswith("4.")]

# ==========================================
# DIMENSION 4: ETL MERGE (Upserts)
# We will increase the volume of updates to 1 Million 
# to overcome the startup overhead of the Low Shuffle algorithm.
# ==========================================

print("Generating 1M UPSERT rows (More volume to show the gap)...")
df_updates = spark.range(0, 1000000) \
    .withColumn("user_id", (F.col("id") % 500000).cast(IntegerType())) \
    .withColumn("session_id", F.rand() * 1000000) \
    .withColumn("url", F.lit("/updated/url/path")) \
    .withColumn("event_date", F.current_date()) \
    .withColumn("event_hour", F.lit(12).cast(IntegerType()))

df_updates.createOrReplaceTempView("source_updates")

# --- TEST A: STANDARD MERGE ---
spark.conf.set("spark.databricks.delta.merge.enableLowShuffle", "false")

def merge_standard():
    # Standard merge will struggle more with 1M rows of updates
    spark.sql("""
        MERGE INTO tbl_liquid t USING source_updates s ON t.id = s.id
        WHEN MATCHED THEN UPDATE SET t.url = s.url
        WHEN NOT MATCHED THEN INSERT *
    """)

run_benchmark("4. ETL Merge", "Standard Shuffle", merge_standard)


# --- TEST B: LOW SHUFFLE MERGE (Modern) ---
spark.conf.set("spark.databricks.delta.merge.enableLowShuffle", "true")

def merge_optimized():
    # Low Shuffle will excel at managing the 1M row merge efficiently
    spark.sql("""
        MERGE INTO tbl_liquid t USING source_updates s ON t.id = s.id
        WHEN MATCHED THEN UPDATE SET t.url = s.url
        WHEN NOT MATCHED THEN INSERT *
    """)

run_benchmark("4. ETL Merge", "Low Shuffle (Modern)", merge_optimized)

# --- SHOW COMPARISON ---
display_results("4.")

## Search

In [0]:
# --- AUTO-CLEANUP ---
global results
results = [r for r in results if not r['Test'].startswith("5.")]

# ==========================================
# DIMENSION 5: SEARCH INDEXING (Bloom Filters)
# Strategy: "The Haystack Problem"
# We create a highly fragmented table (1000 files).
# Standard: Must open/close 1000 files (Slow Metadata/IO).
# Bloom: Checks index, skips 999 files instantly.
# ==========================================

print("Creating a 'Fragmented' table (1000 files) to simulate a Data Lake...")
# Lemos a tabela liquid e a reescrevemos "estilhaçada" em 1000 pedaços
# Isso força o Spark a lidar com muitos arquivos (Metadata Overhead)
df_frag = spark.read.table("tbl_liquid")
df_frag.repartition(1000).write.mode("overwrite").saveAsTable("tbl_fragmented")

# Valor que não existe
missing_session_id = -999999
spark.catalog.clearCache()

# --- TEST A: STANDARD SCAN (No Index) ---
def point_lookup_std():
    # Vai ter que abrir 1000 arquivos para descobrir que o dado não está lá.
    count = spark.read.table("tbl_fragmented").filter(f"session_id = {missing_session_id}").count()
    print(f"Standard scan finished. Found: {count}")

run_benchmark("5. Point Lookup", "Full Scan (Standard)", point_lookup_std)


# --- TEST B: BLOOM FILTER (Indexed) ---
print("🌸 Creating Bloom Filter Index on 'tbl_fragmented'...")
spark.sql("CREATE BLOOMFILTER INDEX ON TABLE tbl_fragmented FOR COLUMNS(session_id)")

def point_lookup_bloom():
    # O Spark vai olhar os índices e pular 99.9% dos arquivos.
    count = spark.read.table("tbl_fragmented").filter(f"session_id = {missing_session_id}").count()
    print(f"Bloom Filter scan finished. Found: {count}")

run_benchmark("5. Point Lookup", "Bloom Filter (Modern)", point_lookup_bloom)

# --- SHOW COMPARISON ---
display_results("5.")

## DevOps

In [0]:
import pyspark.sql.functions as F

# --- AUTO-CLEANUP ---
global results
results = [r for r in results if not r['Test'].startswith("6.")]

# ==========================================
# DIMENSION 6: CLONING (DevOps / CI/CD)
# Scenario: Creating a Staging environment from Production.
# Problem: Small data is too fast to copy. We need HEAVY data to show the pain.
# ==========================================

print("Preparing HEAVY dataset for Cloning test...")

# 1. Create a Heavy Source Table
# We take the existing rows and add a massive text column (payload).
# This bloats the file size, forcing the Deep Clone to work hard copying bytes.
df_heavy_clone = spark.read.table("tbl_liquid") \
    .withColumn("payload", F.expr("repeat('SIMULATING_PRODUCTION_DATA_VOLUME_FOR_BENCHMARK_', 50)"))

# Materialize the source table (Simulating a Production Table)
spark.sql("DROP TABLE IF EXISTS tbl_heavy_source")
df_heavy_clone.write.format("delta").mode("overwrite").saveAsTable("tbl_heavy_source")

print("Tables prepared. Starting Race...")

# --- TEST A: DEEP CLONE (Standard) ---
spark.sql("DROP TABLE IF EXISTS tbl_dev_physical")

def deep_clone():
    # DISADVANTAGE: 'CLONE' (Deep) physically duplicates every byte of the heavy table.
    # It consumes Storage ($$$) and Time (I/O).
    spark.sql("CREATE TABLE tbl_dev_physical CLONE tbl_heavy_source")

run_benchmark("6. Cloning", "Deep Copy (Standard)", deep_clone)


# --- TEST B: SHALLOW CLONE (Modern) ---
spark.sql("DROP TABLE IF EXISTS tbl_dev_virtual")

def shallow_clone():
    # ADVANTAGE: 'SHALLOW CLONE' only copies the Metadata (Transaction Log).
    # It points to the original files. Zero Data Copy. Instant.
    spark.sql("CREATE TABLE tbl_dev_virtual SHALLOW CLONE tbl_heavy_source")

run_benchmark("6. Cloning", "Shallow Clone (Modern)", shallow_clone)

# --- SHOW COMPARISON ---
display_results("6.")

## Final Comparison

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

# --- CONFIGURATION: CUSTOM TITLES ---
# Separando Data Layout em WRITE e READ conforme solicitado
custom_titles = {
    "1. Ingestion":     "Data Layout (Write): Ingestion Cost & Optimization Overhead", 
    "1. Read (Filter)": "Data Layout (Read): Standard vs. Z-Order vs. Liquid Clustering",
    "2. Delete":        "DML Operations: Copy-on-Write vs. Deletion Vectors",
    "3. Caching":       "Read Speed: Cold Read vs. Disk Cache (Hot)",
    "4. ETL Merge":     "ETL Merge: Standard Shuffle vs. Low Shuffle Merge",
    "5. Point Lookup":  "Search: Full Scan vs. Bloom Filters",
    "6. Cloning":       "DevOps: Deep Clone vs. Shallow Clone"
}

# 1. Prepare Data
df_res = pd.DataFrame(results)

# Clean up method names for chart readability
df_res['Method_Short'] = df_res['Method']

# Define Logic for "Standard/Old" vs "New/Optimized" colors
# We treat 'Standard', 'Legacy', 'Cold', and 'Deep' as the baseline (Gray)
df_res['Is_Baseline'] = df_res['Method'].str.contains("Standard") | \
                        df_res['Method'].str.contains("Legacy") | \
                        df_res['Method'].str.contains("Cold") | \
                        df_res['Method'].str.contains("Deep")

# Get unique tests ensuring specific order
ordered_tests = [
    t for t in ["1. Ingestion", "1. Read (Filter)", "2. Delete", "3. Caching", "4. ETL Merge", "5. Point Lookup", "6. Cloning"] 
    if t in df_res['Test'].unique()
]

# 2. Setup Grid Layout (2 Columns)
n_charts = len(ordered_tests)
n_cols = 2
n_rows = math.ceil(n_charts / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows), constrained_layout=True)
axes = axes.flatten()

# Custom Colors: Gray for Old/Standard, Databricks Orange for New/Optimized
palette = {True: "#95a5a6", False: "#ff5722"} # Gray vs Databricks Orange

print(f"--- 📊 DATABRICKS OPTIMIZATION BENCHMARK SUMMARY ---")

# 3. Generate Subplots
for i, test_name in enumerate(ordered_tests):
    ax = axes[i]
    data = df_res[df_res['Test'] == test_name].copy()
    
    # Calculate Baseline for Speedup
    # Logic: Find the row that represents the "Old Way" to set as 1.0x
    try:
        baseline = data[data['Is_Baseline'] == True]['Duration_Sec'].values[0]
    except:
        baseline = data['Duration_Sec'].max() # Fallback if specific baseline not found
        
    data['Speedup'] = baseline / data['Duration_Sec']
    
    # Plot Bar Chart
    sns.barplot(
        data=data, 
        y="Method", 
        x="Duration_Sec", 
        hue="Is_Baseline", 
        ax=ax, 
        palette=palette, 
        dodge=False
    )
    
    # Formatting
    chart_title = custom_titles.get(test_name, test_name)
    
    ax.set_title(chart_title, fontsize=13, fontweight='bold', pad=15)
    ax.set_xlabel("Time (Seconds) - Lower is Better", fontsize=10)
    ax.set_ylabel("")
    
    if ax.get_legend():
        ax.get_legend().remove()
        
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    
    # Add Text Annotations
    for p, row_idx in zip(ax.patches, range(len(data))):
        width = p.get_width()
        if pd.isna(width): continue
            
        speedup = data.iloc[row_idx]['Speedup']
        duration = data.iloc[row_idx]['Duration_Sec']
        
        # Smart Labeling
        if speedup >= 1.1:
            label = f"🚀 {speedup:.1f}x Faster"
            color = "#d35400" # Dark Orange
            fontweight = 'bold'
        elif speedup < 0.9:
            # Highlight slowness (e.g., Z-Order Write Penalty)
            label = f"🔻 {speedup:.1f}x (Slower)"
            color = "#7f8c8d" # Gray
            fontweight = 'normal'
        else:
            label = "1.0x (Baseline)"
            color = "black"
            fontweight = 'normal'
            
        ax.text(
            width + (baseline * 0.02), 
            p.get_y() + p.get_height() / 2, 
            f"{duration:.2f}s  |  {label}", 
            ha='left', va='center', fontsize=11, fontweight=fontweight, color=color
        )

# Hide empty subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.suptitle("Databricks Optimization Suite: Performance Benchmark", fontsize=22, fontweight='bold', y=1.02)
plt.show()